In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.distributions import Normal as norm
from termcolor import colored
from sklearn.decomposition import PCA
from scipy import linalg as LA
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import multinomial
from scipy.stats import logistic
from scipy.stats import expon, uniform, laplace
from Main_functions import *

from My_DML import *
import random
from numpy import genfromtxt
import time

In [2]:
import dml
from dml.dml_eig import DML_eig

from __future__ import print_function, absolute_import
import numpy as np

from six.moves import xrange
from sklearn.utils.validation import check_X_y

from numpy.linalg import inv, eigh
import scipy.linalg as sl

from dml.dml_algorithm import DML_Algorithm
from dml.dml_utils import calc_outers, calc_outers_i

In [3]:
def mean_dist(X, Y, M):
    n = X.shape[0]
    S = 0
    for i in range(n):
        x, y = X[i], Y[i]
        dis_x_y_M = (x-y) @ M @ (x-y)
        S += dis_x_y_M
    return S/n


def pred_M(X, Y, M, t):
    n = X.shape[0]
    Pred = np.zeros(n, dtype = int)
    for i in range(n):
        x, y = X[i], Y[i]
        dis_x_y_M = (x-y) @ M @ (x-y)
        if dis_x_y_M >= t:
            Pred[i] = 1
        else: Pred[i] = 0
    return Pred

def acc(X, Y, D, M, t):
    P = pred_M(X, Y, M, t)
    return (P == D).sum()/X.shape[0]

def best_tau(X, Y, D, M):
    u = 2 * mean_dist(X, Y, M)
    mesh = np.linspace(start = 0, stop = u, num=100)
    best_acc = 0
    best_t = 0
    for t in mesh:
        acc_ = acc(X, Y, D, M, t = t)
        if acc_ > best_acc:
            best_acc = acc_
            best_t = t
    return best_t, best_acc 

### Setting the data distribuation 

Pairs $(x_i,y_i)\in \mathbb{R}^d\times \mathbb{R}^d$ for $i=1, \ldots, N$ are generated i.i.d. from density distribuation $f(\cdot)\times f(\cdot)$.  

Parameters:

$d:$ dimention of data points

$N:$ number of pairs

$f(\cdot):$ density distribuation


### Given the eigenvalues of ground truth $M_t$, we randomly generate it.

In [4]:
csv_path = 'CSVs/logistic_Eig_DML_'
image_path = 'Images/logistic_Eig_DML_'
title_suffix = '\n (Logistic Noise Eig_DML)'
noise_type = 'Logistic'
model_type = 'Logistic'

In [5]:
d = 10
rank = 5
seed = 2023

In [6]:
np.random.seed(seed)
non_zero_eigens_of_M_star = np.round(np.random.uniform(low=0.0, high=1.0, size= rank), 2)

In [7]:
#random.seed(seed)
# non_zero_eigens_of_M_star = random.choices(np.arange(1, 31), k = rank)
# random.uniform()
non_zero_eigens_of_M_star

array([0.32, 0.89, 0.59, 0.13, 0.14])

In [8]:
diag = np.zeros(d)
diag[:rank] = non_zero_eigens_of_M_star
diag

array([0.32, 0.89, 0.59, 0.13, 0.14, 0.  , 0.  , 0.  , 0.  , 0.  ])

In [9]:
data_model = DG(diag, seed = 22)

Ground truth M_t and B_t are generated (M_t = B_t @ B_t.T)
To access to M_t and B_t, use .M_t and .B_t


In [10]:
data_model.M_diag

array([0.32, 0.89, 0.59, 0.13, 0.14, 0.  , 0.  , 0.  , 0.  , 0.  ])

In [11]:
data_model.M_t.shape, data_model.B_t.shape

((10, 10), (10, 10))

### Given the eigenvalue of covariance matrix Cov, we randomly generate it.

In [12]:
np.random.seed(2024)
diag_cov = np.round(np.random.uniform(low=0.0, high=1.0, size= d), 2)

In [13]:
print(diag_cov)

[0.59 0.7  0.19 0.04 0.21 0.11 0.73 0.68 0.47 0.45]


In [14]:
# diag_cov.sort()

In [15]:
print(diag_cov)

[0.59 0.7  0.19 0.04 0.21 0.11 0.73 0.68 0.47 0.45]


In [16]:
diag_cov.sum()

4.17

In [17]:
Sigma, U_sigma = random_covariance(diag_cov, seed = 21)

In [18]:
f = multivariate_normal(mean = np.zeros(d), cov = Sigma)

### What is the average of squared norm of points generating where $x\sim f(x)$ with $\mathbb{E}(x) = \mu$ and ${\rm Cov}(x) = \Sigma$

If $x\sim f(x)$ where $\mathbb{E}(x) = \mu$ and ${\rm Cov}(x) = \Sigma$, then 
\begin{align}
\mathbb{E}\left(x^\top Mx\right) & = \mathbb{E}\left({\rm tr}(x^\top Mx)\right)\\
 & = \mathbb{E}\left({\rm tr}(xx^\top M)\right)\\
 & = {\rm tr}\left(\mathbb{E}\left(xx^\top\right)M\right)\\
 & = {\rm tr}\left(\left[{\rm Cov}(x) - \mathbb{E}(x)\mathbb{E}(x)^\top\right]M\right)
\end{align}

If we set  $\mu = \mathbf{0}$, then 
$$\mathbb{E}\left(x^\top Mx\right) = {\rm tr}\left(\Sigma M\right).$$
Therefore, since $x$ and $y$ are independent, 
$$\mathbb{E}\left(\|x-y\|_M^2\right) = 2{\rm tr}\left(\Sigma M\right).$$

$$\mathbb{E}\left((x-y)^\top (x-y)\right) = \mathbb{E}\left(x^\top x + y^\top y - x^\top y - y^\top x\right) = 2{\rm Cov}(x)$$

In [19]:
snm = 2 * np.trace(data_model.M_t @ Sigma)
print("squared norm mean = {}".format(snm))

squared norm mean = 1.6983720913430767


### Check the average empirically

In [20]:
N = 10000
X_1, Y_1 = sample(f, N)
sn = (((X_1 - Y_1) @ data_model.B_t)**2).sum()/N
print("emperical squared norm mean = {}".format(sn))

emperical squared norm mean = 1.7357384841106784


### Set $\tau$ to generate data points (binary case!)

In [21]:
tau = [1.3]
N = 15000
s = .2

In [22]:
X, Y, D_noisy, D_no_noisy = data_model.generate(f, N, tau = np.array(tau), 
                                                noise_type = noise_type, noise_par = s)

In [23]:
noise_amount = (D_noisy != D_no_noisy).mean()
print('the amount of noise: {}'.format(noise_amount))

the amount of noise: 0.10026666666666667


In [24]:
for i in range(len(tau)+1):
    print('number of pairs with {} as their labels is {}'.format(i, (D_no_noisy==i).sum()))

number of pairs with 0 as their labels is 7586
number of pairs with 1 as their labels is 7414


In [25]:
(D_no_noisy== 0).sum()/N

0.5057333333333334

In [26]:
(D_noisy== 0).sum()/N

0.49293333333333333

In [27]:
def gen_S_D_and_X(X, Y, L):
    S = []
    D = []
    n, d = X.shape
    X_out = np.zeros((2*n, d))
    X_out[:n,:] = X
    X_out[n:,:] = Y
    for i in range(n):
        if L[i] == 1:
            D.append([i, n+i])
        else:
            S.append([i, n+i])
    return X_out, S, D

In [28]:
X_eig, Y_eig, D_noisy_eig, D_no_noisy_eig = X.copy(), Y.copy(), D_noisy.copy(), D_no_noisy.copy()
X_eig_test = X_eig[10000: 15000]
Y_eig_test = Y_eig[10000: 15000]
D_noisy_eig_test = D_noisy_eig[10000: 15000]
D_no_noisy_eig_test = D_no_noisy_eig[10000: 15000]

In [29]:
k = 10
n_labels = len(tau) + 1

In [30]:
X_torch = torch.tensor(X, dtype = torch.float64)
Y_torch = torch.tensor(Y, dtype = torch.float64)
D_noisy_torch = torch.tensor(D_noisy, dtype = torch.torch.int64)
D_no_noisy_torch = torch.tensor(D_no_noisy, dtype = torch.torch.int64)

In [31]:
X_test = X_torch[10000: 15000]
Y_test = Y_torch[10000: 15000]
D_noisy_test = D_noisy_torch[10000: 15000]
D_no_noisy_test = D_no_noisy_torch[10000: 15000]

### Our model

In [36]:
N = 10000
X_train = X_torch[:N]
Y_train = Y_torch[:N]
D_noisy_train = D_noisy_torch[:N]
D_no_noisy_train = D_no_noisy_torch[:N]
t = time.time()
model = ML(d, k, n_labels, 
           X_train, Y_train, D_noisy_train, D_no_noisy_train, 
           X_test, Y_test, D_noisy_test, D_no_noisy_test, 
           Type = model_type)
model.train(learning_rate = 3.5e-1, 
            n_iters = 5001, 
            decay = .95,
            f = f, 
            B_star = data_model.B_t, 
            tau_star = data_model.tau_t, 
            N = 1000,
           show_log = False)
t_1 = time.time() - t
print(' run time = {}\n train_acc_noisy = {}\n train_no_noisy_acc = {}\n'.format(t_1, 
                                                                                model.train_accuracy_with_noise[-1],
                                                                                model.train_accuracy_without_noise[-1])) 

print(' test_acc_noisy = {}\n test_no_noisy_acc = {}\n'.format(model.test_accuracy_with_noise[-1],
                                                               model.test_accuracy_without_noise[-1])) 

 run time = 17.223711013793945
 train_acc_noisy = 0.8997
 train_no_noisy_acc = 0.985

 test_acc_noisy = 0.9058
 test_no_noisy_acc = 0.988



### eig_DML model

In [35]:
N = 10000   
max_it = 25
print('max_it = ', max_it)
X_eig_train = X_eig[:N]
Y_eig_train = Y_eig[:N]
D_noisy_eig_train = D_noisy_eig[:N]
D_no_noisy_eig_train = D_no_noisy_eig[:N]
t = time.time()
X_dml, S, D = gen_S_D_and_X(X_eig_train, Y_eig_train, D_noisy_eig_train)
#     mu = 5e-4

model = My_DML_eig(mu = 1e-4, tol = 1e-5, eps = 1e-10, max_it = max_it)

model.fit(X_dml, S, D)
t_1 = time.time() - t

t_best, best_noisy_train_acc = best_tau(X_eig_train, Y_eig_train, D_noisy_eig_train, 
                                        model.metric())


train_no_noisy_acc = acc(X_eig_train, Y_eig_train, D_no_noisy_eig_train, 
                        model.metric(), t = t_best)




test_noisy_acc = acc(X_eig_test, Y_eig_test, D_noisy_eig_test, 
                        model.metric(), t = t_best)



test_no_noisy_acc = acc(X_eig_test, Y_eig_test, D_no_noisy_eig_test, 
                        model.metric(), t = t_best)

print(' run time = {}\n best_train_acc_noisy = {}\n train_acc_no_noisy = {}\n'.format(t_1,
                                                                                      best_noisy_train_acc,
                                                                                      train_no_noisy_acc))  

print(' test_noisy_acc = {}\n test_no_noisy_acc = {}\n'.format(test_noisy_acc,
                                                               test_no_noisy_acc))  

 run time = 13551.12533712387
 best_train_acc_noisy = 0.8569
 train_acc_no_noisy = 0.897

 test_noisy_acc = 0.8574
 test_no_noisy_acc = 0.8932

